In [100]:
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from keras.preprocessing.image import ImageDataGenerator

In [101]:
# Load the CSV data
data = pd.read_csv('/home/hassan/Downloads/updated_wind_field_estimation_Orb_tnr_spk_TC.csv', delimiter=',',nrows=500)

In [102]:
window_size = 10

In [103]:
print(data)

                                         imageFilename  WindField  geometry_x  \
0    wind_field_estimation_Orb_tnr_spk_TC_images.24...     wind_0  -29.829815   
1    wind_field_estimation_Orb_tnr_spk_TC_images.25...     wind_1  -29.836163   
2    wind_field_estimation_Orb_tnr_spk_TC_images.20...    wind_10  -30.249793   
3    wind_field_estimation_Orb_tnr_spk_TC_images.11...   wind_100  -31.324683   
4    wind_field_estimation_Orb_tnr_spk_TC_images.19...  wind_1000  -32.419873   
..                                                 ...        ...         ...   
495  wind_field_estimation_Orb_tnr_spk_TC_images.52...  wind_1443  -30.008024   
496  wind_field_estimation_Orb_tnr_spk_TC_images.48...  wind_1444  -30.258725   
497  wind_field_estimation_Orb_tnr_spk_TC_images.49...  wind_1445  -30.264941   
498  wind_field_estimation_Orb_tnr_spk_TC_images.49...  wind_1446  -30.130192   
499  wind_field_estimation_Orb_tnr_spk_TC_images.48...  wind_1447  -30.162325   

     geometry_y  speed  dir

In [104]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [105]:
# Load and process the image data for each time step
for i in range(0,data.shape[0]):
    # Load the image data and scale the pixel values
    filename = '/home/hassan/Documents/data/'+data.iloc[i, 0]
    data.iloc[i, 0] = filename

In [106]:
print(data)

                                         imageFilename  WindField  geometry_x  \
0    /home/hassan/Documents/data/wind_field_estimat...     wind_0  -29.829815   
1    /home/hassan/Documents/data/wind_field_estimat...     wind_1  -29.836163   
2    /home/hassan/Documents/data/wind_field_estimat...    wind_10  -30.249793   
3    /home/hassan/Documents/data/wind_field_estimat...   wind_100  -31.324683   
4    /home/hassan/Documents/data/wind_field_estimat...  wind_1000  -32.419873   
..                                                 ...        ...         ...   
495  /home/hassan/Documents/data/wind_field_estimat...  wind_1443  -30.008024   
496  /home/hassan/Documents/data/wind_field_estimat...  wind_1444  -30.258725   
497  /home/hassan/Documents/data/wind_field_estimat...  wind_1445  -30.264941   
498  /home/hassan/Documents/data/wind_field_estimat...  wind_1446  -30.130192   
499  /home/hassan/Documents/data/wind_field_estimat...  wind_1447  -30.162325   

     geometry_y  speed  dir

In [107]:
# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(data.shape[0] * split_ratio)
X_train = data.iloc[:split_index]
X_test = data.iloc[split_index:]

In [108]:
print(X_train)

                                         imageFilename  WindField  geometry_x  \
0    /home/hassan/Documents/data/wind_field_estimat...     wind_0  -29.829815   
1    /home/hassan/Documents/data/wind_field_estimat...     wind_1  -29.836163   
2    /home/hassan/Documents/data/wind_field_estimat...    wind_10  -30.249793   
3    /home/hassan/Documents/data/wind_field_estimat...   wind_100  -31.324683   
4    /home/hassan/Documents/data/wind_field_estimat...  wind_1000  -32.419873   
..                                                 ...        ...         ...   
395  /home/hassan/Documents/data/wind_field_estimat...  wind_1353  -31.347824   
396  /home/hassan/Documents/data/wind_field_estimat...  wind_1354  -31.245117   
397  /home/hassan/Documents/data/wind_field_estimat...  wind_1355  -31.277400   
398  /home/hassan/Documents/data/wind_field_estimat...  wind_1356  -31.251009   
399  /home/hassan/Documents/data/wind_field_estimat...  wind_1357  -31.283281   

     geometry_y  speed  dir

In [109]:
mean_direction = X_train['direction'].mean()
mean_geometry_x = X_train['geometry_x'].mean()
mean_geometry_y = X_train['geometry_y'].mean()
mean_dx = X_train['dx'].mean()
mean_dy = X_train['dy'].mean()
std_direction = X_train['direction'].std()
std_geometry_x = X_train['geometry_x'].std()
std_geometry_y = X_train['geometry_y'].std()
std_dx = X_train['dx'].std()
std_dy = X_train['dy'].std()

In [110]:
print(X_train)

                                         imageFilename  WindField  geometry_x  \
0    /home/hassan/Documents/data/wind_field_estimat...     wind_0  -29.829815   
1    /home/hassan/Documents/data/wind_field_estimat...     wind_1  -29.836163   
2    /home/hassan/Documents/data/wind_field_estimat...    wind_10  -30.249793   
3    /home/hassan/Documents/data/wind_field_estimat...   wind_100  -31.324683   
4    /home/hassan/Documents/data/wind_field_estimat...  wind_1000  -32.419873   
..                                                 ...        ...         ...   
395  /home/hassan/Documents/data/wind_field_estimat...  wind_1353  -31.347824   
396  /home/hassan/Documents/data/wind_field_estimat...  wind_1354  -31.245117   
397  /home/hassan/Documents/data/wind_field_estimat...  wind_1355  -31.277400   
398  /home/hassan/Documents/data/wind_field_estimat...  wind_1356  -31.251009   
399  /home/hassan/Documents/data/wind_field_estimat...  wind_1357  -31.283281   

     geometry_y  speed  dir

In [111]:
X_train['direction'] = (X_train['direction'] - mean_direction) / std_direction
X_train['geometry_x'] = (X_train['geometry_x'] - mean_geometry_x) / std_geometry_x
X_train['geometry_y'] = (X_train['geometry_y'] - mean_geometry_y) / std_geometry_y
X_train['dx'] = (X_train['dx'] - mean_dx) / std_dx
X_train['dy'] = (X_train['dy'] - mean_dy) / std_dy
X_test['direction'] = (X_test['direction'] - mean_direction) / std_direction
X_test['geometry_x'] = (X_test['geometry_x'] - mean_geometry_x) / std_geometry_x
X_test['geometry_y'] = (X_test['geometry_y'] - mean_geometry_y) / std_geometry_y
X_test['dx'] = (X_test['dx'] - mean_dx) / std_dx
X_test['dy'] = (X_test['dy'] - mean_dy) / std_dy

In [112]:
print(X_train)

                                         imageFilename  WindField  geometry_x  \
0    /home/hassan/Documents/data/wind_field_estimat...     wind_0    1.675968   
1    /home/hassan/Documents/data/wind_field_estimat...     wind_1    1.667612   
2    /home/hassan/Documents/data/wind_field_estimat...    wind_10    1.123125   
3    /home/hassan/Documents/data/wind_field_estimat...   wind_100   -0.291819   
4    /home/hassan/Documents/data/wind_field_estimat...  wind_1000   -1.733486   
..                                                 ...        ...         ...   
395  /home/hassan/Documents/data/wind_field_estimat...  wind_1353   -0.322281   
396  /home/hassan/Documents/data/wind_field_estimat...  wind_1354   -0.187082   
397  /home/hassan/Documents/data/wind_field_estimat...  wind_1355   -0.229577   
398  /home/hassan/Documents/data/wind_field_estimat...  wind_1356   -0.194838   
399  /home/hassan/Documents/data/wind_field_estimat...  wind_1357   -0.237319   

     geometry_y  speed  dir

In [113]:
train_images = train_generator.flow_from_dataframe(
    dataframe=X_train,
    x_col='imageFilename',
    y_col=['direction', 'speed'],
    target_size=(256, 256),
    class_mode='raw',
    batch_size=32,
    subset='training'
)

Found 320 validated image filenames.


In [114]:
val_images = train_generator.flow_from_dataframe(
    dataframe=X_train,
    x_col='imageFilename',
    y_col=['direction','speed'],
    target_size=(256, 256),
    class_mode='raw',
    batch_size=32,
    subset='validation'
)

Found 80 validated image filenames.


In [115]:
test_imges = test_generator.flow_from_dataframe(
    dataframe=X_test,
    x_col='imageFilename',
    y_col=['direction','speed'],
    target_size=(256, 256),
    class_mode='raw',
    batch_size=32,
)

Found 100 validated image filenames.


In [116]:
inputs = tf.keras.Input(shape=(256, 256, 3))

# Convert the input shape from (batch_size, 120, 120, 3) to (batch_size, sequence_length, input_dim)
x = tf.keras.layers.Reshape(target_shape=(256, 256*3))(inputs)

# Replace the Conv2D and MaxPool2D layers with recurrent layers
x = tf.keras.layers.LSTM(units=32, return_sequences=True, activation='tanh')(x)
x = tf.keras.layers.LSTM(units=32, return_sequences=True, activation='tanh')(x)

x = tf.keras.layers.LSTM(units=64, return_sequences=True, activation='tanh')(x)
x = tf.keras.layers.LSTM(units=64, return_sequences=False, activation='tanh')(x)

x = tf.keras.layers.Dense(128, activation='tanh')(x)
x = tf.keras.layers.Dense(128, activation='tanh')(x)

outputs = tf.keras.layers.Dense(2, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


In [117]:
# Compile the model
model.compile(optimizer='adam', loss='mse',metrics='accuracy')

In [118]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
10/10 [==============================] - 11s 484ms/step - loss: 94.4844 - accuracy: 0.8906 - val_loss: 33.7202 - val_accuracy: 0.9750
Epoch 2/100
10/10 [==============================] - 4s 369ms/step - loss: 14.8422 - accuracy: 0.9906 - val_loss: 13.8364 - val_accuracy: 0.9750
Epoch 3/100
10/10 [==============================] - 4s 350ms/step - loss: 9.8368 - accuracy: 0.9906 - val_loss: 14.5620 - val_accuracy: 0.9750
Epoch 4/100
10/10 [==============================] - 3s 348ms/step - loss: 9.1824 - accuracy: 0.9906 - val_loss: 12.6596 - val_accuracy: 0.9750
Epoch 5/100
10/10 [==============================] - 3s 346ms/step - loss: 8.7912 - accuracy: 0.9906 - val_loss: 12.0907 - val_accuracy: 0.9750
Epoch 6/100
10/10 [==============================] - 4s 376ms/step - loss: 8.8734 - accuracy: 0.9906 - val_loss: 12.1523 - val_accuracy: 0.9750
Epoch 7/100
10/10 [==============================] - 3s 349ms/step - loss: 8.8271 - accuracy: 0.9906 - val_loss: 12.2944 - val_accura

In [119]:
# Evaluate the model on the test data
test_loss = model.evaluate(test_imges)

# Print the test loss
print("Test Loss:", test_loss)


4/4 [==============================] - 1s 124ms/step - loss: 11.7734 - accuracy: 0.9800
Test Loss: [11.773416519165039, 0.9800000190734863]
